## Biogas plant

You want to plan the two-year supply of raw materials for a biogas power plant. Such a plant produces energy by burning biogas, which is obtained from the bacterial fermentation of organic wastes. 
Specifically, your plant is powered by corn chopping, a residual of agro-industrial operations that you can purchase from 5 local farms. 
The table below shows the quarterly capacity of each farm for the next two years. Quantities are measured in tons.

Farm|T1|T2|T3|T4|T5|T6|T7|T8
:-|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:
1|700|1500|700|0|0|700|1500|0
2|1350|0|450|0|1350|0|450|0
3|0|1500|1500|0|0|1500|1500|0
4|820|1560|820|0|820|1560|820|0
5|0|680|1080|0|0|680|1080|0

Due to crop rotations and corn harvesting periods, farms are unable to supply material in some quarters. Moreover the types of corn chopping provided are different, each coming with its own unitary purchase price, unitary storage cost and percentage of dry matter. The table below shows a summary of these information.

Farm|Purchase price|Storage cost|Dry matter
:-|:-:|:-:|:-:
1|0.20|0.002|15
2|0.18|0.012|28
3|0.19|0.007|35
4|0.21|0.011|37
5|0.23|0.015|42

Your biogas plant must operate by burning a mixture of corn choppings with a dry matter percentage between 20% and 40%. Under these conditions, the yield is 421.6 kWh of energy per ton of burned material. The energy produced by the plant is sold on the market at a price of 0.28 $/kWh. 

Due to state regulations, all biogas plants can produce a maximum of 1950 MWh of energy per quarter. You are allowed to store corn chopping in a silo, whose total capacity is of 500 tons. 

Plan the supply and inventory of your biogas plant with the goal of maximizing your profits (i.e., revenues minus costs).

In [ ]:
# When using Colab, make sure you run this instruction beforehand
!pip install --upgrade cffi==1.15.0
import importlib
import cffi
importlib.reload(cffi)
!pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 446 kB 4.9 MB/s 
  Attempting uninstall: cffi
    Found existing installation: cffi 1.15.1
    Uninstalling cffi-1.15.1:
      Successfully uninstalled cffi-1.15.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 4.8 MB/s 


### **Sets and Parameters**

In [ ]:
from IPython.utils.tempdir import TemporaryDirectory
import mip

#Quarter capacity [tons]
C = \
[[700, 1500, 700, 0, 0, 700, 1500, 0],
 [1350, 0, 450, 0, 1350, 0, 450, 0],
 [0, 1500, 1500, 0, 0, 1500, 1500, 0],
 [820, 1560, 820, 0, 820, 1560, 820, 0],
 [0, 680, 1080, 0, 0, 680, 1080, 0]]


#unitary purchase price [$/tons]
p = [0.20, 0.18, 0.19, 0.21, 0.23]

#unitary storage cost [$/tons]
s =[0.002, 0.012, 0.007, 0.011, 0.015]

#percentage of dry matter
d = [0.15, 0.28, 0.35, 0.37, 0.42]

#yield [KWh/tons]
b = 421.6

#energy sold price [$/KWh]
r = 0.28

#max energy per quarter [KWh]
max_E = 1950 * 1000

#max capacity of silo [tons]
max_C = 500

f = len(C)
F = range (f)
t =len(C[0])
T = range (t)
T1 = range(t-1)

### Constraints

In [ ]:
m = mip.Model()

#material purchased 
x = [[m.add_var() for j in T] for i in F]

#inventory level
y = [[m.add_var() for j in T] for i in F]

#material burned
z = [[m.add_var() for j in T] for i in F]

#cannot exceed the capacity of the silo

[m.add_constr(mip.xsum(z[i][j] for i in F) <= max_C) for j in T]

#cannot produce more than max_E
[m.add_constr(b * mip.xsum(y[i][j] for i in F) <= max_E) for j in T]

#cannot purchase more than what has been produced
[[m.add_constr(x[i][j] <= C[i][j]) for j in T] for i in F]

#constraint on the percentage of dry matter 
for j in T:
  m.add_constr(mip.xsum(y[i][j] * d[i] for i in F) <= 0.4 * mip.xsum(y[i][j] for i in F))
  m.add_constr(mip.xsum(y[i][j] * d[i] for i in F) >= 0.2 * mip.xsum(y[i][j] for i in F))

#constraint on the storage level

for i in F:
  m.add_constr((0 + x[i][0]) == (z[i][0] + y[i][0])) #Condition when the storage is on the first quarter
  for j in T1:  
      m.add_constr((z[i][j] + x[i][j+1]) == (z[i][j+1] + y[i][j+1]))

### Objective Function

In [ ]:
#maximize the profit over the 8 quarters
m.objective = mip.maximize(mip.xsum( 
    (r * b * mip.xsum(y[i][j] for i in F)) - 
    (mip.xsum(p[i]*x[i][j] for i in F) + mip.xsum (s[i]*z[i][j] for i in F))
    for j in T))

### Results

In [ ]:
m.optimize()

print("x:")
for i in F:
  for j in T:
    print('%.f'%(x[i][j]), sep = "\t", end = " ")
  print()


print("\n\nobjective:")
print(m.objective_value)

x:
700 1500 700 0 0 700 1500 0 
1350 0 450 0 1350 0 450 0 
0 1500 1500 0 0 1500 1500 0 
820 1560 820 0 820 1560 820 0 
0 565 1080 0 0 680 855 0 


objective:
2861373.925412714
